This script gathers price data from Aave API
https://aave-api-v2.aave.com/

In [ ]:
pip install requests

In [1]:
import requests
import pandas as pd
import datetime
from datetime import timedelta
import time

In [73]:
#pool = '0xb53c1a33016b2dc2ff3653530bff1848a515c8c5' v2
pool = '0x24a42fd28c976a61df5d00d0599c34c4f90748c8' #v1
date = datetime.datetime(2020, 12, 2)
date_string = date.strftime('%m-%d-%Y')

In [65]:
lst = []

In [93]:
def process_aave_v1(raw):
    for i in range(0,len(raw)):
        temp = []
        if raw[i]['symbol'] == 'USDC':
            epoch = ''
            time = date_string
            lend = float(raw[i]['avg1DaysLiquidityRate'])
            borrow = float(raw[i]['avg1DaysVariableBorrowRate'])
            temp.append(['usdc', epoch, time, lend, borrow])
        elif raw[i]['symbol'] == 'USDT':
            epoch = ''
            time = date_string
            lend = float(raw[i]['avg1DaysLiquidityRate'])
            borrow = float(raw[i]['avg1DaysVariableBorrowRate'])
            temp.append(['usdt', epoch, time, lend, borrow])
        elif raw[i]['symbol'] == 'DAI':
            epoch = ''
            time = date_string
            lend = float(raw[i]['avg1DaysLiquidityRate'])
            borrow = float(raw[i]['avg1DaysVariableBorrowRate'])
            temp.append(['dai', epoch, time, lend, borrow])
        elif raw[i]['symbol'] == 'ETH':
            epoch = ''
            time = date_string
            lend = float(raw[i]['avg1DaysLiquidityRate'])
            borrow = float(raw[i]['avg1DaysVariableBorrowRate'])
            temp.append(['eth', epoch, time, lend, borrow])
        elif raw[i]['symbol'] == 'WBTC':
            epoch = ''
            time = date_string
            lend = float(raw[i]['avg1DaysLiquidityRate'])
            borrow = float(raw[i]['avg1DaysVariableBorrowRate'])
            temp.append(['wbtc', epoch, time, lend, borrow])
            
        else:
            continue
            
        lst.append(temp)

In [52]:
def process_aave(raw):
    for i in range(0,len(raw)):
        temp = []
        if raw[i]['symbol'] == 'USDC':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            lend = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            temp.append(['usdc', epoch, time, lend, borrow])
        elif raw[i]['symbol'] == 'USDT':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            lend = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            temp.append(['usdt', epoch, time, lend, borrow])
        elif raw[i]['symbol'] == 'DAI':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            lend = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            dai = [epoch, time, lend, borrow]
            temp.append(['dai', epoch, time, lend, borrow])
        elif raw[i]['symbol'] == 'WETH':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            lend = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            dai = [epoch, time, lend, borrow]
            temp.append(['weth', epoch, time, lend, borrow])
        elif raw[i]['symbol'] == 'WBTC':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            lend = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            dai = [epoch, time, lend, borrow]
            temp.append(['wbtc', epoch, time, lend, borrow])
            
        else:
            continue
            
        lst.append(temp)

In [75]:
cont = True

while cont:
    query = f'https://aave-api-v2.aave.com/data/liquidity/v2?poolId={pool}&date={date_string}'
    response = requests.get(query)
    raw = response.json()
    process_aave_v1(raw)
    date = (date - timedelta(days=1))
    date_string = date.strftime('%m-%d-%Y')
    print(date_string)
    time.sleep(3)


12-01-2020
11-30-2020


KeyboardInterrupt: 

In [88]:
x = [item for sublist in lst[2::5] for item in sublist]

In [89]:
df_aave = pd.DataFrame(x, columns=['asset', 'epoch', 'timestamp', 'lend', 'borrow'])
df = df_aave.copy()
df


,asset,epoch,timestamp,lend,borrow
0,usdc,,12-02-2020,0.015262,0.037109
1,usdc,,12-01-2020,0.015262,0.037109
2,usdc,,11-30-2020,0.015262,0.037109
3,usdc,,11-29-2020,0.015262,0.037109
4,usdc,,11-28-2020,0.015262,0.037109
...,...,...,...,...,...
334,usdc,,01-03-2020,0.015262,0.037109
335,usdc,,01-02-2020,0.015262,0.037109
336,usdc,,01-01-2020,0.014217,0.035705
337,usdc,,12-02-2020,0.015262,0.037109


In [44]:
df = df.sort_values(by='timestamp', ascending=True)
df = df.reset_index(drop=True)

In [45]:
#store in csv
df.to_csv(f'aave_usdt_interest.csv', index=False)